In [1]:
!pip install openprompt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 3.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 32.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 5.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from openprompt.plms import load_plm

plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-cased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [2]:
from openprompt.data_utils import InputExample

classes = ["negative", "positive"]
dataset = [
    # text_a is the input text of the data, some other datasets may have multiple input sentences in one example.
    InputExample(
        guid = 0,
        text_a = "Albert Einstein was one of the greatest intellects of his time.",
    ),
    InputExample(
        guid = 1,
        text_a = "The film was badly made.",
    )
]

In [13]:
from openprompt.prompts import ManualTemplate

prompt_template = ManualTemplate(text = '{"placeholder":"text_a"} It was {"mask"}', tokenizer=tokenizer)

In [14]:
prompt_template.text

[{'add_prefix_space': '', 'placeholder': 'text_a'},
 {'add_prefix_space': ' ', 'text': 'It was'},
 {'add_prefix_space': ' ', 'mask': None}]

In [15]:
from openprompt.prompts import ManualVerbalizer

prompt_verbalizer = ManualVerbalizer(
    classes=classes,
    label_words={
        "negative": ["bad"],
        "positive": ["good", "wonderful", "great"]
    },
    tokenizer=tokenizer
)

In [16]:
from openprompt import PromptForClassification

prompt_model = PromptForClassification(
    template=prompt_template,
    plm=plm,
    verbalizer=prompt_verbalizer
)

In [17]:
prompt_model

PromptForClassification(
  (prompt_model): PromptModel(
    (plm): BertForMaskedLM(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(28996, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
                (output): BertSelfOutput(
          

In [18]:
from openprompt import PromptDataLoader
data_loader = PromptDataLoader(
    dataset=dataset,
    tokenizer=tokenizer,
    template=prompt_template,
    tokenizer_wrapper_class=WrapperClass
)

tokenizing: 2it [00:00, 1301.97it/s]


In [22]:
import torch

prompt_model.eval()
with torch.no_grad():
    for batch in data_loader:
        logits = prompt_model(batch)
        preds = torch.argmax(logits, dim=-1)
        print(batch)
        print(classes[preds])

{"input_ids": [[101, 3986, 16127, 1108, 1141, 1104, 1103, 4459, 1107, 7854, 18465, 1116, 1104, 1117, 1159, 119, 1135, 1108, 103, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [25]:
data_loader.raw_dataset

[{
   "guid": 0,
   "label": null,
   "meta": {},
   "text_a": "Albert Einstein was one of the greatest intellects of his time.",
   "text_b": "",
   "tgt_text": null
 },
 {
   "guid": 1,
   "label": null,
   "meta": {},
   "text_a": "The film was badly made.",
   "text_b": "",
   "tgt_text": null
 }]

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "KETI-AIR/ke-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/599 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/944M [00:00<?, ?B/s]

In [3]:
type(model)

transformers.models.t5.modeling_t5.T5ForConditionalGeneration

In [5]:
input_str = "klue_mrc question: {question} context: {article}"
input_data = tokenizer(input_str, return_tensors="pt")

In [8]:
model(input_ids=input_data.input_ids, attention_mask=input_data.attention_mask)

ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds